###  This is the master company addition file.
####  It adds entries to the nedbank_companies collection in lookubs db on the MongoDb

In [6]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')

In [7]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [8]:
import re
import pandas as pd
from NedbankSpendPy import  client
import time
import numpy as np
from collections import Counter
from datetime import datetime

Assign the subclass_id.  Check if it exists.  If it doesn't, it needs to be added using new_classification.js
Find the appropriate subclass_id in "Classifications Standard Industrial Classification of all Economic Activities
(SIC) Seventh Edition" here (http://www.statssa.gov.za/classifications/codelists/Web_SIC7a/SIC_7_Final_Manual_Errata.pdf)

In [9]:
nedbank_companies_df=pd.DataFrame(list(client.lookups['nedbank_companies'].find({})))
subclass_id_list=[56101,47110,47610,45500,47620]
generic_avoidlist=[]
for subclass_id in subclass_id_list:
    for aliases in nedbank_companies_df[nedbank_companies_df.subclass_id==subclass_id].aliases:
        for alias in aliases:
#             print(alias)
            generic_avoidlist.append(alias)

In [10]:
generic_avoidlist.append('NANDOS')
generic_avoidlist.index('NANDOS')
generic_avoidlist.append('OCEAN BASKET')
generic_avoidlist.append('BEAUTY')
generic_avoidlist.append('SERVICE STATION')
generic_avoidlist.append('KauaiCascades')
generic_avoidlist.append('WimpyCabanas')
generic_avoidlist.append('QUEENSPARK')
generic_avoidlist.append('DALPARK PLAZA')
generic_avoidlist.append('BAR')
generic_avoidlist.append('RESTA')
# DALPARK PLAZA
generic_avoidlist=list(set(generic_avoidlist))

In [11]:
subclass_id = 55101 # look carefully at this assignment!
companyname='TRAVELSTART'; # Assign the companyname.  This how it is found by other scripts!
groupname='TRAVEL'; # Assign the group name:
aliases=['TRAVELSTART'];# Define the alias list. This is IMPORTANT. 
# It is the key code that will find this entity in the clientswipes db 
# It is the actual string that will search the franchise names
generic_avoidlist.append('NANDOS');# Define the anti-aliases, if NO antialias is necessary leave the list EMPTY, NOT and empty string:
channel = 0, # bricks and mortar == 0; mixed == 1; online == 2
discretionary = 2; #non-discretionary == 0; mixed == 1; discretionary == 2
period='201907'
start_date=datetime(2019,7,1)
end_date=datetime(2019,7,31)
generic_avoidlist=list(set(generic_avoidlist))

Construct the regex string here:

In [18]:
avoidlist=[]
aliases=['BOLT','TAXIFY']
period='201907'
start_date=datetime(2019,7,1)
end_date=datetime(2019,7,5)

In [19]:
regex_str=mongo_regex_searchstring(aliases=aliases,avoidlist=avoidlist)
regex_str_compiled = re.compile(regex_str, re.IGNORECASE)
print(regex_str, regex_str_compiled)

Construct regex string ...
aliases =  ['BOLT', 'TAXIFY']
regex_alias_str =  BOLT|TAXIFY
regex_avoid_str =  
(?m)^.*BOLT|TAXIFY.*$ re.compile('(?m)^.*BOLT|TAXIFY.*$', re.IGNORECASE|re.MULTILINE)


Check if the names exist, this may take a minute or two!

### The following snippet search franchisenames for the regular expression string defined by regex_str.
``` 
It gives out put that serves as a guide
```

In [ ]:
collection=client.behaviourdb['clientswipes_{period}'.format(period=period)]

find_dict= {
    'TransactionAmount':{'$lt':0},
    'TransactionDate':
        {
            '$gte':start_date,
            '$lt':end_date
        },
    'franchisename': 
        {
            '$regex': re.compile(regex_str, re.IGNORECASE)
        },
    'companyname': 
        {
            '$exists': False
        }    
    }


df=pd.DataFrame(list(collection.find(find_dict)))

df.shape
franchisenamelist = list(set(df.franchisename))
franchisenamelist.sort(reverse = True)
franchisenamelist

In [ ]:
df.shape

Look at the unique return values:

In [ ]:
franchisenamelist

Look at the contents of the result:

In [ ]:
df

In [ ]:
result_df=df[['franchisename','TransactionAmount']].groupby('franchisename').sum()
result_df.reset_index(inplace=True)
result_df.sort_values('TransactionAmount',inplace=True)
result_df.TransactionAmount.cumsum()

In [ ]:
result_df

### Are you OK with the above results?
#### If yes, then construct the input dictionary like so, paying careful attention to the output:

In [ ]:
find_regex_str = regex_searchstring(aliases, avoidlist)

In [ ]:
input_dict = input_dict_fn(companyname=companyname,
                           avoidlists=avoidlist,
                           subclass_id = subclass_id,
                  aliases = aliases,
                  channel = channel,
                  discretionary = discretionary,
                  groupname=groupname,
                          regex_str=find_regex_str)

```
Look CAREFULLY at the output below.  This is what is going into the db.  It will affect everything downstream
```

In [ ]:
input_dict

```
The following code inserts it into the db:
```

In [ ]:
nedbank_companies_db.insert_one(input_dict)

```
The following code now adds the regex string to the company entry to the company record.  It does this for ALL documents.
```

In [ ]:
new_searchdf = pd.DataFrame(list(client.lookups['nedbank_companies'].find()))
for index, row in new_searchdf.iterrows():
    regex_str = mongo_regex_searchstring_row(row)
    results = client.lookups['nedbank_companies'].update_one({ '_id': row['_id'] }, {'$set': {'regex_str':regex_str}} )
    print('modified: ',results.modified_count, 'for ', row["companyname"])

## Here we do an analysis of transaction found in the clientswipes db

In [ ]:
collection=client.behaviourdb['clientswipes_{period}'.format(period=period)]

In [ ]:
nedbank_companies_df=pd.DataFrame(list(client.lookups['nedbank_companies'].find({})))
nedbank_companies_df.to_csv('/home/christo/data/nedbank_companies_df.csv')

In [ ]:
list(nedbank_companies_df)

In [ ]:
df=nedbank_companies_df[nedbank_companies_df.subclass_id==subclass_id]

In [ ]:
subclass_id

In [ ]:
def find_dict_fn(regex_str, start_date, end_date):
    return {
    'TransactionDate':
        {
            '$gte':start_date,
            '$lt':end_date
        },
    'franchisename': 
        {
            '$regex': re.compile(regex_str, re.IGNORECASE)
        },
#     'companyname': 
#         {
#             '$exists': False
#         }    
    }
# return find_dict

In [ ]:
df[df.companyname=="SUN CITY"].regex_str.values[0]

In [ ]:
regex_str=df[df.companyname=="SUN CITY"].regex_str.values[0]
avoidlist=df[df.companyname=="SUN CITY"].avoidlists.values[0]
aliases=df[df.companyname=="SUN CITY"].aliases.values[0]

In [ ]:
avoidlist.index('NANDOS')

In [ ]:
output = pd.DataFrame()
df=nedbank_companies_df[nedbank_companies_df.subclass_id==subclass_id]
for regex_str, companyname, groupname in zip(list(df.regex_str), list(df.companyname), list(df.groupname)):
    print('companyname: ',companyname,', groupname: ',groupname)
#     print(regex_str,companyname,groupname)
    find_dict = find_dict_fn(regex_str, start_date, end_date)
    success=False
    count=0
    while not success and count<100:
        try:
            count+=1
            print(count)
            find_df=pd.DataFrame(list(collection.find(find_dict)))
            if not find_df.empty:
                franchisenamelist = list(set(find_df.franchisename))    
                for franchisename in franchisenamelist:
                    gf=find_df[find_df.franchisename==franchisename]
                    print('count: ',count,', companyname: ',companyname,', groupname: ',groupname, ', franchisename ',franchisename,
                          ', total_amount: ', sum(gf.TransactionAmount),
                         ', total_count: ',gf.shape[0])
                    df_dict={'companyname':companyname,
                             'groupname':groupname,
                             'franchisename':franchisename,
                             'total_amount':sum(gf.TransactionAmount),
                            'total_count':gf.shape[0]}
                    output = output.append(df_dict, ignore_index=True)
            success=True
        except Exception as e:
            print('Error in with message {e}'.format(e=e))
            time.sleep(60)


In [ ]:
output.shape

In [ ]:
output.to_csv(path_or_buf='/home/christo/data/hotel_franchise_output_v3.csv')

### In what follows we ADD a generic avoid list to the existing avoid list of each company in df:

In [ ]:
subclass_id

In [ ]:
nedbank_companies_df=pd.DataFrame(list(client.lookups['nedbank_companies'].find({})))
df=nedbank_companies_df[nedbank_companies_df.subclass_id==subclass_id]
for companyname in list(df.companyname):
    print(companyname)
    l=list(client.lookups['nedbank_companies'].find({'companyname':companyname}))
    doc=l[0]
    newdoc=doc
    newdoc['avoidlists']=list(set(doc['avoidlists']+generic_avoidlist))
    find_regex_str = regex_searchstring(newdoc['aliases'], newdoc['avoidlists'])
    client.lookups['nedbank_companies'].update_one({'_id':newdoc['_id']},{'$set':{'avoidlists':newdoc['avoidlists'],
                                                                                 'regex_str':find_regex_str},
                                                                          '$unset':{'(?m)^(?!':1,'title':1}})

In [ ]:
generic_avoidlist

In [ ]:
generic_avoidlist.index('BAR')